In [ ]:
from pathlib import Path
from sqlalchemy import create_engine

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

# Se define la ruta para la base de datos local SQLite
project_name = Path.cwd().parent.parent.name
data_projetc_path = Path.home().joinpath(f'Dropbox/La Casa Del Carpintero/Departamento de Programación/data_projects_git/data_{project_name}')
db_file = 'comisiones.db'
db_file_path_str = str(data_projetc_path.joinpath(db_file))

# Se crea el objeto engine para trabajarlo con los metodos de pandas
engine = create_engine(f'sqlite:///{db_file_path_str}')

In [ ]:
file01 = 'clientes_regalos'
file01_path_str = str(data_projetc_path.joinpath(f'comisiones/{file01}.xlsx'))
clientes_regalos = pd.read_excel(file01_path_str, dtype_backend='numpy_nullable')

file02 = 'refacturaciones'
file02_path_str = str(data_projetc_path.joinpath(f'costo_ventas/{file02}.xlsx'))
refacturaciones = pd.read_excel(file02_path_str, dtype_backend='numpy_nullable')

In [ ]:
with engine.connect() as conn, conn.begin():

    cobranza_año = pd.read_sql_table('cobranza_año_2024', conn, dtype_backend='numpy_nullable')

    costo_ventas_enero = pd.read_sql_table('costo_ventas_1_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_febrero = pd.read_sql_table('costo_ventas_2_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_marzo = pd.read_sql_table('costo_ventas_3_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_abril = pd.read_sql_table('costo_ventas_4_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_mayo = pd.read_sql_table('costo_ventas_5_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_junio = pd.read_sql_table('costo_ventas_6_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_julio = pd.read_sql_table('costo_ventas_7_2024', conn, dtype_backend='numpy_nullable')
    
    datos_com_2023 = pd.read_sql_table('datos_com_2023', conn, dtype_backend='numpy_nullable')
    bonos_limites_comisiones = pd.read_sql_table('bonos_limites_comisiones', conn, dtype_backend='numpy_nullable')

engine.dispose()

costo_ventas_list = [costo_ventas_enero, costo_ventas_febrero, costo_ventas_marzo, costo_ventas_abril, costo_ventas_mayo, costo_ventas_junio, costo_ventas_julio]
costo_ventas_año = pd.concat(costo_ventas_list)

In [ ]:
def gerentes_list_func() -> list:

    gerentes_list = [
                    'Carlos Manuel Guzmán Rios',
                    'Mariana Araceli Carvajal Flores',
                    'Rosario Martinez Zarate'
                    ]

    return gerentes_list

In [ ]:
def comisiones_mes_2024_func(mes:int, costo_ventas_list:list, bonos_limites_comisiones:pd.DataFrame, gerentes_list:list) -> pd.DataFrame:

    df = costo_ventas_list[mes-1]
    mes_df = df['invoice_date'].dt.month.unique()[0]

    df['mes'] = mes_df


    comisiones_2024_vend = (df.loc[
        ~df['salesperson_name'].isin(gerentes_list)
        ,
                [
                    'salesperson_id',
                    'salesperson_name',
                    'business_model',
                    'mes',
                    'price_subtotal',
                    'utilidad_partida_$_after_cc'
                ]
            ].groupby('salesperson_id')
            .agg({
                'salesperson_name':['first'], 
                'business_model':['first'], 
                'mes':['first'], 
                'price_subtotal':['sum'],
                'utilidad_partida_$_after_cc':['sum']
                })
            .set_axis(
                [
                    'salesperson_name',
                    'business_model',
                    'mes',
                    'facturacion',
                    'utilidad'
                ], axis='columns')
            .reset_index()
    )


    comisiones_2024_ger = pd.DataFrame(
                [
                    [206, 'Mariana Araceli Carvajal Flores','Piso', mes_df],
                    [208, 'Carlos Manuel Guzmán Rios','CE', mes_df],
                    [209, 'Rosario Martinez Zarate', 'Piso', mes_df],
                ],
                columns = 
                    [
                        'salesperson_id',
                        'salesperson_name',
                        'business_model',
                        'mes'
                    ]
            )

    comisiones_2024_ger_values = (df.loc[:,
                [
                    'sale_team_description',
                    'mes',
                    'price_subtotal',
                    'utilidad_partida_$_after_cc'
                ]
            ].groupby('sale_team_description')
            .agg({
                'mes':['first'], 
                'price_subtotal':['sum'],
                'utilidad_partida_$_after_cc':['sum']
                })
            .set_axis(
                [
                    'mes',
                    'facturacion',
                    'utilidad'
                ], axis='columns')
            .reset_index()
    )


    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Carlos Manuel Guzmán Rios'
            , 
            ['facturacion', 'utilidad']
        ] = (comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'].isin(['A1 CE', 'A2 CE'])
                ]
                .groupby('mes')
                .sum()
                [['facturacion', 'utilidad']]
                .values
            )

    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Mariana Araceli Carvajal Flores'
            , 
            ['facturacion', 'utilidad']
        ] = comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'] == 'A1 Piso'
                ][['facturacion', 'utilidad']].values

    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Rosario Martinez Zarate'
            , 
            ['facturacion', 'utilidad']
        ] = comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'] == 'A2 Piso'
                ][['facturacion', 'utilidad']].values


    comisiones_2024 = pd.concat([comisiones_2024_ger, comisiones_2024_vend], ignore_index=True)


    for i in range(len(comisiones_2024)):

        if comisiones_2024.iloc[i]['salesperson_name'] in gerentes_list:
            
            bonos_limites_comisiones_line = bonos_limites_comisiones.loc[
                (bonos_limites_comisiones['business_model'] == comisiones_2024.iloc[i]['business_model'])
                & (bonos_limites_comisiones['is_gerente'] == True)
                & (bonos_limites_comisiones['lim_inf'] <= comisiones_2024.iloc[i]['utilidad'])
                & (bonos_limites_comisiones['lim_sup'] >= comisiones_2024.iloc[i]['utilidad'])
            ]

        else:
            
            bonos_limites_comisiones_line = bonos_limites_comisiones.loc[
                (bonos_limites_comisiones['business_model'] == comisiones_2024.iloc[i]['business_model'])
                & (bonos_limites_comisiones['is_gerente'] == False)
                & (bonos_limites_comisiones['lim_inf'] <= comisiones_2024.iloc[i]['utilidad'])
                & (bonos_limites_comisiones['lim_sup'] >= comisiones_2024.iloc[i]['utilidad'])
            ]

        comisiones_2024.loc[i, ['comision_%']] = bonos_limites_comisiones_line[['%']].iloc[0].values
        comisiones_2024.loc[i, ['comision_$']] = comisiones_2024.loc[i, ['comision_%']].iloc[0] * comisiones_2024.loc[i, ['utilidad']].iloc[0]
        comisiones_2024.loc[i, ['bono_venta_$']] = bonos_limites_comisiones_line[['bono']].iloc[0].values


    return comisiones_2024

In [ ]:
def comision_cobranza_2024_func(cobranza_año:pd.DataFrame, datos_com_2023:pd.DataFrame, comisiones_2024:pd.DataFrame,  gerentes_list:list) -> pd.DataFrame:

    cobranza_año.loc[:, ['fact_año']] = cobranza_año['fact_doc_date'].dt.year
    cobranza_año.loc[:, ['fact_mes']] = cobranza_año['fact_doc_date'].dt.month
    cobranza_año.loc[:, ['mes']] = cobranza_año['pay_date'].dt.month

    comision_cobranza = cobranza_año.loc[
                                cobranza_año['paga_comision'] == True,
                                [
                                    'salesperson_id',
                                    'salesperson_name',
                                    'mes',
                                    'pay_amount',
                                    'fact_año',
                                    'fact_mes'
                                ]
                            ].groupby(
                                ['salesperson_id', 
                                'salesperson_name', 
                                'mes', 
                                'fact_año', 
                                'fact_mes'
                                ]).sum().reset_index()

    for i in range(len(comision_cobranza)):

        año = comision_cobranza.iloc[i]['fact_año']

        if año == 2022 or año == 2023:
            venta_comision_line = datos_com_2023.loc[
                                        (datos_com_2023['salesperson_id'] == comision_cobranza.iloc[i]['salesperson_id'])
                                        & (datos_com_2023['año'] == comision_cobranza.iloc[i]['fact_año'])
                                        & (datos_com_2023['mes'] == comision_cobranza.iloc[i]['fact_mes'])
                                    ]
        
        else:
            venta_comision_line = comisiones_2024.loc[
                                        (comisiones_2024['salesperson_id'] == comision_cobranza.iloc[i]['salesperson_id'])
                                        & (comisiones_2024['mes'] == comision_cobranza.iloc[i]['fact_mes'])
                                    ]
        
        comision_cobranza.loc[i, ['facturacion']] = venta_comision_line[['facturacion']].iloc[0].values
        comision_cobranza.loc[i, ['cobranza_%']] = (comision_cobranza.loc[i, ['pay_amount']].iloc[0] / comision_cobranza.loc[i, ['facturacion']].iloc[0]) * 100
        comision_cobranza.loc[i, ['comision_mes']] = venta_comision_line[['comision_$']].iloc[0].values
        comision_cobranza.loc[i, ['comision_cobranza_$']] = comision_cobranza.loc[i, ['cobranza_%']].iloc[0] / 100 * comision_cobranza.loc[i, ['comision_mes']].iloc[0]

    return comision_cobranza

In [ ]:
def comision_pagos_2024_func(comisiones_2024:pd.DataFrame, comision_cobranza_2024:pd.DataFrame) -> pd.DataFrame:

    comision_pagos = comision_cobranza_2024[[
                                'salesperson_id', 
                                'salesperson_name', 
                                'mes', 
                                'comision_cobranza_$'
                            ]].groupby([
                                'salesperson_id', 
                                'salesperson_name', 
                                'mes', 
                            ]).sum().reset_index().merge(
                                comisiones_2024[[
                                    'salesperson_id', 
                                    'mes', 
                                    'bono_venta_$'
                                ]],
                                how='left',
                                on=['salesperson_id', 'mes']
                            )

    comision_pagos.loc[
            ~comision_pagos[ 'bono_venta_$'].isna()
            ,
            'total_comision'
        ] = comision_pagos['comision_cobranza_$'] + comision_pagos['bono_venta_$'] * 0.88

    comision_pagos.loc[
        comision_pagos[ 'bono_venta_$'].isna()
            ,
            'total_comision'
        ] = comision_pagos['comision_cobranza_$'] * 0.88


    return comision_pagos

In [ ]:
gerentes_list = gerentes_list_func()

comisiones_1_2024 = comisiones_mes_2024_func(1, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_2_2024 = comisiones_mes_2024_func(2, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_3_2024 = comisiones_mes_2024_func(3, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_4_2024 = comisiones_mes_2024_func(4, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_5_2024 = comisiones_mes_2024_func(5, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_6_2024 = comisiones_mes_2024_func(6, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_7_2024 = comisiones_mes_2024_func(7, costo_ventas_list, bonos_limites_comisiones, gerentes_list)

comisiones_2024 = pd.concat([
                        comisiones_1_2024,
                        comisiones_2_2024,
                        comisiones_3_2024,
                        comisiones_4_2024,
                        comisiones_5_2024,
                        comisiones_6_2024,
                        comisiones_7_2024
                    ])

comision_cobranza_2024 = comision_cobranza_2024_func(cobranza_año, datos_com_2023, comisiones_2024, gerentes_list)

comision_pagos_2024 = comision_pagos_2024_func(comisiones_2024, comision_cobranza_2024)

In [ ]:
comision_pagos_2024[['salesperson_name', 'total_comision']].groupby('salesperson_name').sum()

In [ ]:
# with engine.connect() as conn, conn.begin():  
#     comisiones_2024.to_sql(name = 'comisiones_2024', index= False, con = conn, if_exists = 'replace')
#     comision_cobranza_2024.to_sql(name = 'comision_cobranza_2024', index= False, con = conn, if_exists = 'replace')

# engine.dispose()

# print('Todas las tablas se ingresaron con éxito')